In [2]:
from embeddings.demo import Demo
import spacy

d = Demo('data/result_1/best_model.pt', 'data/result_1/saved_config.json', 'data/rel_20.txt')
nlp = spacy.load('en_core_web_sm')

Loaded checkpoint 'data/result_1/best_model.pt' (epoch 18 iter: 690001 train_loss: 1.6085507930369372, dev_loss: 2.29362431439209, train_pos:0.5143910646438599, train_neg: 0.01834769733250141, dev_pos: 0.28918400406837463, dev_neg: 0.015907999128103256)


In [2]:
from rouge import rouge_n_sentence_level
def sent_normalize(sent:str):
    tokens = nlp(sent)
    normalized = []
    for token in tokens:
        if token.pos_ == 'ADV':
            continue
        else:
            normalized.append(token.lemma_)
    return ' '.join(normalized)

def my_rouge(a:str,b:str):
    a_ = a.split()
    b_ = b.split()
    summary, reference = (a_, b_) if len(a_)>=len(b_) else (b_, a_)
    score = rouge_n_sentence_level(summary, reference, n=1)
    return score.recall, ' '.join(reference)

def move_redundency(sents:list):
    ret = []
    for sent in sents:
        if not ret:
            ret.append(sent)
            continue
        is_redundent = False
        i = 0
        while i < len(ret):
            ret_sent = ret[i]
            score, shorter_sent = my_rouge(ret_sent, sent)
            if score >= 0.7:
                if ret_sent == shorter_sent:
                    is_redundent = True
                    break
                else:
                    ret.remove(ret_sent)
            else:
                i += 1
        if not is_redundent:
            ret.append(sent)
    return ret

In [14]:
# Get the top matched relations
subject = 'linear regression'.split()
object = 'data mining'.split()
display_num = 60

result = d.find_closest_relation(subject, object, display_num)
result = [line.split('<pad>', 1)[0] for line in result]

In [15]:
# Remove redundent relations and showing the similarity
clean_result_raw = move_redundency(result)
score = d.cal_score(subject, object, clean_result_raw)
for item in zip(clean_result_raw, score):
    print(item)

('are popular tools for ', 2.8260436)
('are suboptimal for ', 2.7709877)
('are powerful methods for ', 2.6435244)
('been applied for ', 2.6390243)
('are technique for ', 2.5502396)
('been applied in ', 2.3141925)
('are approach for ', 2.3084748)
('are used for ', 2.2900667)
('is widely employed for ', 2.2461405)
('is often used in ', 2.2136934)
('are method for ', 2.190134)
('are more suitable for ', 2.1745002)
('has been widely used for ', 2.1741714)
('was used in ', 2.1066484)
('be widely used in ', 2.082003)
('are widely adopted in ', 2.0813704)
('is widely used to ', 2.0643475)
('are successfully used in ', 2.0634754)
('be effectively used for ', 2.0526202)
('is commonly used for ', 2.052291)
('are tasks for ', 2.049424)


In [16]:
# Remove redundent relations after lemmatization
normalized_result = [sent_normalize(sent) for sent in result]
clean_result = move_redundency(normalized_result)
print('\n'.join(clean_result))

be use for
be popular tool for
be suboptimal for
be method for
be suitable for
be apply for
be technique for
use in
be apply in
be approach for
be employ for
be adopt in
be use to
be task for


In [30]:
rel_1 = 'are analogues of'.split()
rel_2 = 'is investigated under'.split()
rel_1 += ['<pad>'] * (6 - len(rel_1))
rel_2 += ['<pad>'] * (6 - len(rel_2))
temp_rel = d.model.get_relation([rel_1, rel_2])
a = temp_rel[0]
b = temp_rel[1]
b.dot(b)

tensor(50.9102)